# Paso1: Seleccion y preparacion del Dataset

In [2]:
import pandas as pd
import requests
import os

# URLs de los archivos
urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-03.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-04.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-05.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-06.parquet"
]

# Función para descargar archivo
def download_file(url, filename):
    print(f"Descargando {filename}...")
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"✅ {filename} descargado")

# Descargar todos los archivos
for url in urls:
    filename = url.split('/')[-1]  # Extrae el nombre del archivo
    if not os.path.exists(filename):
        download_file(url, filename)
    else:
        print(f"⏭️  {filename} ya existe, saltando...")

print("🎉 Todas las descargas completadas!")

Descargando yellow_tripdata_2016-01.parquet...
✅ yellow_tripdata_2016-01.parquet descargado
Descargando yellow_tripdata_2016-02.parquet...
✅ yellow_tripdata_2016-02.parquet descargado
Descargando yellow_tripdata_2016-03.parquet...
✅ yellow_tripdata_2016-03.parquet descargado
Descargando yellow_tripdata_2016-04.parquet...
✅ yellow_tripdata_2016-04.parquet descargado
Descargando yellow_tripdata_2016-05.parquet...
✅ yellow_tripdata_2016-05.parquet descargado
Descargando yellow_tripdata_2016-06.parquet...
✅ yellow_tripdata_2016-06.parquet descargado
🎉 Todas las descargas completadas!
